<br>

<img src="https://fedlex.data.admin.ch/assets/logo.png" style="width:25%; float:right">

# Fedlex Tutorial

This page serves as a introductory tutorial to the [Fedlex Linked Data ecosystem](https://fedlex.data.admin.ch).



# Introduction

The webpage you are currently viewing is a so called **interactive JupyterLite notebook**. In this notebook, you can change interactively the content of the single cells and execute these cells directly seeing the result of your changes immediately. The cells contain either [Markdown](https://en.wikipedia.org/wiki/Markdown) content (like this cell) or executable Python source code.

JupyterLite stems from JupyterLab with the advantage of being completely browser based without any backend infrastructure. This means that the execution of the cells could take some time during first execution. Subsequent executions will be much faster because of stored data in your browser cache.

If you are unfamiliar with the handling of Jupyter notebooks, here are two useful resources:

- [The JupyterLab Interface](https://jupyterlab.readthedocs.io/en/stable/user/interface.html)
- [The Jupyter Notebook](https://jupyterlab.readthedocs.io/en/stable/user/notebook.html)

# Preliminaries

In this part some preliminaries for querying LINDAS with SPARQL are introduced. If you are only interested in the actual LINDAS tutorial, you can skip the whole section and start [here](#Actual-Tutorial).

## Install and Import the Necessary Modules

Querying a SPARQL endpoint is basically making a POST request to the corresponding endpoint URL. As JupyterLite at the moment has no support for Python's `requests` modul, the JavaScript fetch API is used (with some tricks). To make this happen, the following modules have to be importet: 

In [2]:
%pip install -q folium

In [3]:
import json
import pandas as pd
from pyodide.ffi import to_js
from IPython.display import JSON, HTML
from js import Object, fetch
from io import StringIO

## Define Main Query Function

As the JavaScript fetch API is asynchronous, the corresponding Python function `query` has to be declared as `async`. This function allows to query the 3 Swiss governmental triple stores.

In [4]:
async def query(query_string, store = "L"):
    
    # three Swiss triplestores
    if store == "F":
        address = 'https://fedlex.data.admin.ch/sparqlendpoint'
    elif store == "G":
        address = 'https://geo.ld.admin.ch/query'
    else:
        address = 'https://ld.admin.ch/query'
    
    # try the Post request with help of JS fetch
    # the creation of the request header is a little bit complicated because it needs to be a 
    # JavaScript JSON that is made within a Python source code
    try:
        resp = await fetch(address,
          method="POST",
          body="query=" + query_string,
          credentials="same-origin",
          headers=Object.fromEntries(to_js({"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8", 
                                            "Accept": "text/csv" })),
        )
    except:
        raise RuntimeError("fetch failed")
    
    
    if resp.ok:
        res = await resp.text()
        # ld.admin.ch throws errors starting with '{"message":'
        if '{"message":' in res:
            error = json.loads(res)
            raise RuntimeError("SPARQL query malformed: " + error["message"])
        # geo.ld.admin.ch throws errors starting with 'Parse error:'
        elif 'Parse error:' in res:
            raise RuntimeError("SPARQL query malformed: " + res)
        else:
            # if everything works out, create a pandas dataframe from the csv result
            df = pd.read_csv(StringIO(res))
            return df
    else:
        # fedlex.data.admin.ch throws error with response status 400
        if resp.status == 400:
            raise RuntimeError("Response status 400: Possible malformed SPARQL query. No syntactic advice available.")
        else:
            raise RuntimeError("Response status " + resp.status)

If you are interested in the details of using the JavaScript fetch API within JupyterLite, please consult:

- https://pyodide.org/en/stable/usage/faq.html#how-can-i-use-fetch-with-optional-arguments-from-python
- https://github.com/jupyterlite/jupyterlite/discussions/412
- https://lwebapp.com/en/post/pyodide-fetch

## Define Display Function

Displays pandas dataframe resulting from the SPARQL query as HTML with clickable links.

In [5]:
def display_result(df):
    df = HTML(df.to_html(render_links=True, escape=False))
    display(df)

# Tutorial

## Vokabular

### ConceptScheme

Ein guter Weg um mit den Fedlex Daten vertraut zu werden ist, sich das verwendete Vokabular genauer anzuschauen. Das Vokabular ist mit Hilfe von [skos:ConceptScheme](http://www.w3.org/2004/02/skos/core#ConceptScheme) zusammengefasst und tritt in Fedlex typischerweise auf der Objekt-Position innerhalb eines Triples auf. Es kann also mit Hilfe von SPARQL nach solchen *ConceptSchemes* gesucht werden:

In [6]:
df = await query("""

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX purl: <http://purl.org/dc/terms/>

SELECT DISTINCT * WHERE {
    ?conceptScheme a skos:ConceptScheme;
        purl:title ?title.
FILTER(lang(?title) = "de")
} 

""", "F")

display_result(df)

,conceptScheme,title
0,https://fedlex.data.admin.ch/vocabulary/consultation-stage,Arbeitsschritte eines Vernehmlassungsverfahrens
1,https://fedlex.data.admin.ch/vocabulary/consultation-status,Stand des Vernehmlassungsverfahrens
2,https://fedlex.data.admin.ch/vocabulary/country,Staaten
3,https://fedlex.data.admin.ch/vocabulary/eutext-subject-theme,Klassifizierung der sektoriellen Abkommen mit der Europäischen Union
4,https://fedlex.data.admin.ch/vocabulary/impact-type,Betroffenheiten
5,https://fedlex.data.admin.ch/vocabulary/information-source,Art der Datenquelle
6,https://fedlex.data.admin.ch/vocabulary/instrument-type,Art des Rechtsaktes
7,https://fedlex.data.admin.ch/vocabulary/international-actor,Internationale Organisationen
8,https://fedlex.data.admin.ch/vocabulary/legal-analysis-status,Stand der Erfassung der Betroffenheiten
9,https://fedlex.data.admin.ch/vocabulary/legal-institution,Organisationseinheiten


### Inhalt eines einzelnen ConceptSchemes

Wenn die Inhalte eines einzelnen ConecptSchemes abgefragt werden soll, kann folgende SPARQL Query helfen:

In [7]:
df = await query("""

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX purl: <http://purl.org/dc/terms/>

SELECT DISTINCT * WHERE {
    ?concept skos:inScheme <https://fedlex.data.admin.ch/vocabulary/treaty-status>;
        skos:prefLabel ?label.
    FILTER(lang(?label) = "de")
} 

""", "F")

display_result(df)

,concept,label
0,https://fedlex.data.admin.ch/vocabulary/treaty-status/1,abgeschlossen (drafts)
1,https://fedlex.data.admin.ch/vocabulary/treaty-status/2,in Kraft
2,https://fedlex.data.admin.ch/vocabulary/treaty-status/3,unterzeichnet
3,https://fedlex.data.admin.ch/vocabulary/treaty-status/4,provisorische Anwendung
4,https://fedlex.data.admin.ch/vocabulary/treaty-status/5,abgelaufen
5,https://fedlex.data.admin.ch/vocabulary/treaty-status/6,ratifiziert


### Dereferenziertes Vokabular

Das gesamte Vokabular kann auch über https://fedlex.data.admin.ch/vocabularies direkt als Webpage abgerufen werden. Diese Website ist hierarchisch aufgebaut und wenn auf https://fedlex.data.admin.ch/vocabulary/treaty-status geklickt wird, erhält man alle Infos zu diesem spezifischen skos:ConceptScheme als Website. Analog gilt dies auch für die einzelnen skos:Concept (bspw. https://fedlex.data.admin.ch/vocabulary/treaty-status/1).

## Aufbau der Daten

Die Website https://fedlex.admin.ch basiert auf Daten aus dem RDF Triple Store. Diese Daten sind auch über das SPARQL Interface oder per SPARQL Query an den Endpunkt verfügbar. Informationen, die man aus den HTML Seiten von Fedlex rauslesen kann, sind also auch maschinenlesbar via SPARQL query verfügbar. Nachfolgend soll Schritt für Schritt das Datenmodell von Fedlex eingeführt werden und die entsprechenden SPARQL Queries aufgezeigt werden, um auf diese Daten maschinenlesbar zugreifen zu können.

### Sammlungen

Die Fedlex Daten sind in drei verschiedene Sammlungen gruppiert:

* Bundesblatt ([Erläuterungen](https://www.fedlex.admin.ch/de/fga/explanation-fg))
* Amtliche Sammlung ([Erläuterungen](https://www.fedlex.admin.ch/de/oc/explanations-oc))
* Systematische Sammlung ([Erläuterungen](https://www.fedlex.admin.ch/de/cc/explanations-cc))

Für juristisch wenig bewanderte Nutzerinnen und Nutzer sind entsprechende Erläuterungen zu den einzelnen Sammlungen verlinkt.

## Beispiel Bundesverfassung

Der Einstieg in die Linked Data Welt von Fedlex geschieht über die URI eines Dokuments. Die URI ist dabei ein eindeutiger Identifier, der einer URL gleicht. Am einfachsten ist diese URI über das Webinterface von Fedlex herauszufinden: 

<br><img src="img/fedlex_search.jpg" width="500px">

<br>Nachdem man die Seite für die Bundesverfassung aufgerufen hat, versteckt sich die URI hinter dem Ketten-Symbol direkt neben "Alles einblenden":

<br><img src="img/sr101.jpg" width="500px">

Damit ist nun klar, dass die URI der Bundesverfassung https://fedlex.data.admin.ch/eli/cc/1999/404 ist. Ein Klick auf diese URI führt zu einer Weiterleitung auf https://www.fedlex.admin.ch/eli/cc/1999/404/de, was der HTML Darstellung der Bundesverfassung in der aktuellen Version entspricht. Man beachte den Unterschied zwischen URI und der Webadresse der HTML Darstellung. 

Um weitere Daten zur Bundesverfassung zu erhalten, gibt es vier Möglichkeiten:

- die HTML Version unter https://www.fedlex.admin.ch/eli/cc/1999/404/de
- die Anzeige der weiteren Daten zur URI mit Hilfe des Metadatenviewers unter https://fedlex.data.admin.ch/de-CH/metadata; diese Seite kann auch direkt mit der zu untersuchenden URI aufgerufen werden, bspw. https://fedlex.data.admin.ch/de-CH/metadata?value=https://fedlex.data.admin.ch/eli/cc/1999/404
- die Abfrage der weiteren Daten via SPARQL Interface unter https://fedlex.data.admin.ch/de-CH/sparql
- die Abfrage der weiteren Daten via SPARQL Query über den SPARQL Endpoint und weiterer Verarbeitung im aktuellen Notebook

Im weiteren Verlauf des Tutorials soll die zweite und vierte Variante benutzt werden:

### Weitere Daten zur Bundesverfassung

Die ersten SPARQL Queries sollen die Information aus der Metadatendarstellung unter https://fedlex.data.admin.ch/de-CH/metadata reproduzieren. Zuerst die Literals:

In [25]:
df = await query("""

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX purl: <http://purl.org/dc/terms/>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT * WHERE {
    <https://fedlex.data.admin.ch/eli/cc/1999/404> ?eigenschaft ?wert.
    FILTER(isLITERAL(?wert) = True)
} 

""", "F")

display_result(df)

,eigenschaft,wert
0,http://data.legilux.public.lu/resource/ontology/jolux#parliamentDraftId,1996.091
1,http://purl.org/dc/terms/identifier,19995395
2,http://data.legilux.public.lu/resource/ontology/jolux#historicalLegalId,101
3,http://data.legilux.public.lu/resource/ontology/jolux#dateEntryInForce,2000-01-01
4,http://data.legilux.public.lu/resource/ontology/jolux#dateDocument,1999-04-18
5,http://cogni.internal.system/model#firstPublicationDate,1999-09-28
6,http://cogni.internal.system/model#skipIndex,0


Anschliessend die ausgehenden Beziehungen:

In [20]:
df = await query("""

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX purl: <http://purl.org/dc/terms/>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT * WHERE {
    <https://fedlex.data.admin.ch/eli/cc/1999/404> ?eigenschaft ?wert.
    FILTER(isURI(?wert) = True)
} 

""", "F")

display_result(df)

,eigenschaft,wert
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#Work
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#WorkAtOj
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#ConsolidationAbstract
3,http://data.legilux.public.lu/resource/ontology/jolux#basicAct,https://fedlex.data.admin.ch/eli/oc/1999/404
4,http://data.legilux.public.lu/resource/ontology/jolux#typeDocument,https://fedlex.data.admin.ch/vocabulary/resource-type/55
5,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/de
6,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/en
7,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/fr
8,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/it
9,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/rm


Und zuletzt noch die einkommenden Beziehungen:

In [21]:
df = await query("""

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX purl: <http://purl.org/dc/terms/>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT * WHERE {
    ?wert ?eigenschaft <https://fedlex.data.admin.ch/eli/cc/1999/404>.
} 

""", "F")

display_result(df)

,wert,eigenschaft
0,https://fedlex.data.admin.ch/eli/dl/proj/2021/44/cons_1,http://data.legilux.public.lu/resource/ontology/jolux#foreseenImpactToLegalResource
1,https://fedlex.data.admin.ch/eli/dl/proj/6000/15/cons_1,http://data.legilux.public.lu/resource/ontology/jolux#foreseenImpactToLegalResource
2,https://fedlex.data.admin.ch/eli/dl/proj/6000/18/cons_1,http://data.legilux.public.lu/resource/ontology/jolux#foreseenImpactToLegalResource
3,https://fedlex.data.admin.ch/eli/dl/proj/6007/49/cons_1,http://data.legilux.public.lu/resource/ontology/jolux#foreseenImpactToLegalResource
4,https://fedlex.data.admin.ch/eli/dl/proj/6007/64/cons_1,http://data.legilux.public.lu/resource/ontology/jolux#foreseenImpactToLegalResource
5,https://fedlex.data.admin.ch/eli/dl/proj/6009/132/cons_1,http://data.legilux.public.lu/resource/ontology/jolux#foreseenImpactToLegalResource
6,https://fedlex.data.admin.ch/eli/dl/proj/6010/53/cons_1,http://data.legilux.public.lu/resource/ontology/jolux#foreseenImpactToLegalResource
7,https://fedlex.data.admin.ch/eli/dl/proj/6011/6/cons_1,http://data.legilux.public.lu/resource/ontology/jolux#foreseenImpactToLegalResource
8,https://fedlex.data.admin.ch/eli/dl/proj/6012/139/cons_1,http://data.legilux.public.lu/resource/ontology/jolux#foreseenImpactToLegalResource
9,https://fedlex.data.admin.ch/eli/dl/proj/6014/122/cons_1,http://data.legilux.public.lu/resource/ontology/jolux#foreseenImpactToLegalResource


### Die Bundesverfassung als PDF

Als nächstes Ziel soll ein Link auf die PDF Datei der Bundesverfassung abgefragt werden. Dazu muss etwas tiefer ins Datenmodell eingestigen werden. Ein Blick auf die ausgehenden Verbindungen der Bundesverfassung lässt die Vermutung zu, dass sich evt. unter der URI https://fedlex.data.admin.ch/eli/cc/1999/404/de etwas verbergen würden. Ein Klick auf die URI (die einem automatisch zum Metadatenviewer bringt), zeigt aber auf, dass dies eher eine Sackgasse ist. Interessanter sind die Links [jolux:isMemberOf](http://data.legilux.public.lu/resource/ontology/jolux#isMemberOf)

### URI Schema

Die eigentlichen Daten (nämlich Gesetzestexte) sind nach einem Schema mit URI versehen. Dieses [URI Schema]https://fedlex.data.admin.ch/de-CH/home/convention richtet sich nach dem System der [European Legislation Identifier (ELI)](https://eur-lex.europa.eu/eli-register/about.html).

Grundsätzlich beginnen alle Daten mit folgender URI: http://fedlex.data.admin.ch/eli

### Bundesblatt

Folgende Query gibt die aktuelle Ausgabe des Bundesblatts zurück:

In [9]:
df = await query("""

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX purl: <http://purl.org/dc/terms/>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT * WHERE {
    ?fga jolux:isMemberOf* <https://fedlex.data.admin.ch/eli/collection/fga>;
        jolux:publicationDate ?date.
} ORDER BY desc(?date) LIMIT 1

""", "F")

display_result(df)

,fga,date
0,https://fedlex.data.admin.ch/eli/collection/fga/2023/36,2023-02-22


### Amtliche Sammlung

In einem weiteren Schritt geht es darum, den Aufbau der Daten unter https://fedlex.data.admin.ch besser zu verstehen. Ein guter Einstiegspunkt ist die Suche unter https://fedlex.admin.ch. Wenn wir dort in der systematischen Rechtssammlung nach der Bundesverfassung suchen, gelangen wir auf die Seite https://www.fedlex.admin.ch/eli/oc/1999/404. Das ist auch gerade die URI der Bundesverfassung.

Wir können nun eine SPARQL Query schreiben, mit der wir danach suchen wollen, wo die Bundesverfassung überall auf der Subjekt-Position vorkommt, um weiter ins Datenmodell einzutauchen. Wir fragen also danach, welche Metadaten zur Bundesverfassung gespeichert sind:

In [10]:
df = await query("""

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT *  WHERE {
  <https://fedlex.data.admin.ch/eli/oc/1999/404> ?p ?o.
} 

""", "F")

display_result(df)

,p,o
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#Work
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#WorkAtOj
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#Act
3,http://purl.org/dc/terms/created,1999-10-26T00:00:00
4,http://purl.org/dc/terms/modified,1999-10-26T00:00:00
5,http://data.legilux.public.lu/resource/ontology/jolux#legalResourceGenre,https://fedlex.data.admin.ch/vocabulary/legal-resource-genre/100
6,http://data.legilux.public.lu/resource/ontology/jolux#processType,https://fedlex.data.admin.ch/vocabulary/type-procedure/ord
7,http://data.legilux.public.lu/resource/ontology/jolux#responsibilityOf,https://fedlex.data.admin.ch/vocabulary/legal-institution/5
8,http://data.legilux.public.lu/resource/ontology/jolux#sequenceInTheYearOfPublication,404
9,http://data.legilux.public.lu/resource/ontology/jolux#typeDocument,https://fedlex.data.admin.ch/vocabulary/resource-type/55


### Fassungen

In [11]:
df = await query("""

PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
 
SELECT ?fassung ?date ?pdf WHERE {
    
    ?fassung jolux:isMemberOf <https://fedlex.data.admin.ch/eli/cc/1999/404>;
        jolux:dateApplicability ?date;
        jolux:isRealizedBy ?realization.
    ?realization jolux:language <http://publications.europa.eu/resource/authority/language/DEU>;
        jolux:isEmbodiedBy ?embodyment.
    ?embodyment jolux:format <http://publications.europa.eu/resource/authority/file-type/PDF>;
        jolux:isExemplifiedBy ?pdf.
        
} ORDER BY DESC(?date)

""", "F")

display_result(df)

,fassung,date,pdf
0,https://fedlex.data.admin.ch/eli/cc/1999/404/20220213,2022-02-13,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20220213/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20220213-de-pdf-a-4.pdf
1,https://fedlex.data.admin.ch/eli/cc/1999/404/20211128,2021-11-28,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20211128/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20211128-de-pdf-a-2.pdf
2,https://fedlex.data.admin.ch/eli/cc/1999/404/20210307,2021-03-07,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20210307/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20210307-de-pdf-a-3.pdf
3,https://fedlex.data.admin.ch/eli/cc/1999/404/20210101,2021-01-01,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20210101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20210101-de-pdf-a-2.pdf
4,https://fedlex.data.admin.ch/eli/cc/1999/404/20200101,2020-01-01,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20200101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20200101-de-pdf-a.pdf
5,https://fedlex.data.admin.ch/eli/cc/1999/404/20180923,2018-09-23,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20180923/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20180923-de-pdf-a.pdf
6,https://fedlex.data.admin.ch/eli/cc/1999/404/20180101,2018-01-01,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20180101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20180101-de-pdf-a.pdf
7,https://fedlex.data.admin.ch/eli/cc/1999/404/20170924,2017-09-24,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20170924/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20170924-de-pdf-a.pdf
8,https://fedlex.data.admin.ch/eli/cc/1999/404/20170212,2017-02-12,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20170212/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20170212-de-pdf-a.pdf
9,https://fedlex.data.admin.ch/eli/cc/1999/404/20160101,2016-01-01,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20160101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20160101-de-pdf-a.pdf


### Zitate

In [12]:
df = await query("""

PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
 
SELECT DISTINCT ?srNr ?title ?article ?citation  WHERE {
    
    ?text jolux:legalResourceSubdivisionIsPartOf <https://fedlex.data.admin.ch/eli/cc/1991/857_857_857> . # Gehe vom consolidationAbstract zum text
    ?text jolux:legalResourceSubdivisionType <https://fedlex.data.admin.ch/vocabulary/subdivision-type/text> . # Nimm nur die Subdivisions vom Typ Text (dort sind die citations)
    ?citation jolux:citationToLegalResource ?text . # Finde alle citations die auf den gewünschten Text zeigen
    ?citation jolux:descriptionTo ?article .# Finde die Beschreibung auf welchen Teil des Textes verlinkt wird
    FILTER(lang(?article) = "de") # Wähle nur deutsche Beschreibungen aus (Das Citation Objekt existiert für jede Sprache separat)
    
    ?citation jolux:citationFromLegalResource/jolux:legalResourceSubdivisionIsPartOf/jolux:isMemberOf ?citedBY .# walk the path from the LegalResourceSubdivision -> Consolidation -> ConsolidationAbstract
    ?citedBY jolux:classifiedByTaxonomyEntry/skos:notation ?srNr . # get SR-Number from consolodationAbstract
    ?citedBY jolux:isRealizedBy ?expression . # Wähle alle Expressions (Sprachausgaben)
    ?expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> . # Wähle nur die deutsche Expression
    ?expression jolux:title ?title . # Wähle den Titel
} ORDER BY asc(?srNr)

""", "F")

display_result(df)

,srNr,title,article,citation
0,142.205,Verordnung vom 15. August 2018 über die Integration von Ausländerinnen und Ausländern (VIntA),Art. 18 Berichterstattung und Kontrolle zu kantonalen Integrationsprogrammen,https://fedlex.data.admin.ch/eli/cc/2018/511/text/20190501/citation/11
1,142.205,Verordnung vom 15. August 2018 über die Integration von Ausländerinnen und Ausländern (VIntA),Art. 14 Kantonale Integrationsprogramme,https://fedlex.data.admin.ch/eli/cc/2018/511/text/20190501/citation/3
2,142.205,Verordnung vom 15. August 2018 über die Integration von Ausländerinnen und Ausländern (VIntA),Art. 16 Übrige Beiträge zugunsten kantonaler Integrationsprogramme,https://fedlex.data.admin.ch/eli/cc/2018/511/text/20190501/citation/9
3,142.312,"Asylverordnung 2 vom 11. August 1999 über Finanzierungsfragen (Asylverordnung 2, AsylV 2)",Art. 68 Bundesbeiträge an die Kantone,https://fedlex.data.admin.ch/eli/cc/1999/360/text/20210101/citation/166
4,151.31,"Verordnung vom 19. November 2003 über die Beseitigung von Benachteiligungen von Menschen mit Behinderungen (Behindertengleichstellungsverordnung, BehiV)",Art. 8,https://fedlex.data.admin.ch/eli/cc/2003/668/text/20210101/citation/22
5,151.34,Verordnung vom 12. November 2003 über die behindertengerechte Gestaltung des öffentlichen Verkehrs (VböV),Art. 24 Auszahlung und Rückforderung,https://fedlex.data.admin.ch/eli/cc/2003/669/text/20201101/citation/17
6,172.010.1,Regierungs- und Verwaltungsorganisationsverordnung vom 25. November 1998 (RVOV),Art. 6 Grundsätze,https://fedlex.data.admin.ch/eli/cc/1999/170/text/20210101/citation/117
7,172.010.1,Regierungs- und Verwaltungsorganisationsverordnung vom 25. November 1998 (RVOV),Art. 27 Überprüfung von Bundesaufgaben,https://fedlex.data.admin.ch/eli/cc/1999/170/text/20210101/citation/183
8,173.110.0,Verordnung vom 15. Januar 1992 über die teilweise Inkraftsetzung der Änderung des Bundesgesetzes über die Organisation der Bundesrechtspflege,Art. 2,https://fedlex.data.admin.ch/eli/cc/1992/337_337_337/text/19921201/citation/17
9,194.2,Bundesgesetz vom 5. Oktober 2007 zur Förderung der Information über den Unternehmensstandort Schweiz,Art. 4 Abgeltungen und Finanzhilfen,https://fedlex.data.admin.ch/eli/cc/2008/75/text/20210101/citation/8
